In [3]:
from pymatgen.core import Structure
#from pymatgen.core.structure import Structure
from pymatgen.core.periodic_table import get_el_sp
from maml.apps.bowsr.model.megnet import MEGNet
from maml.apps.bowsr.model.cgcnn import CGCNN
from maml.apps.bowsr.optimizer import BayesianOptimizer

structures = []

# import required module
import os
# assign directory
directory = 'EndPOSCARs'
 
# iterate over files in
# that directory
index = 0
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        structures.append(Structure.from_file(f))
        if(index % 1000 == 0):
            print(index)
        index += 1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [4]:
from aviary.cgcnn.utils import get_cgcnn_input
from aviary.wren.utils import count_wyks, get_aflow_label_spglib
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [5]:
import pandas as pd
def expected_radius(struct):
    element_list = struct.composition.chemical_system.split('-')
    element_list = [get_el_sp(e) for e in element_list]
    ele1, ele2 = sorted(element_list, key=lambda x: x.atomic_radius)[:2]
    return ele1.atomic_radius + ele2.atomic_radius

In [6]:
model = MEGNet() # or `model = CGCNN()`
import os, sys
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

def relaxStructure(structure):
    try:
        compressed_optimizer = BayesianOptimizer(
            model=model,
            structure=structure,
            relax_coords=True,
            relax_lattice=True,
            use_symmetry=False
        )

        compressed_optimizer.set_bounds()
        compressed_optimizer.optimize(n_init=80, n_iter=80, alpha=0.026 ** 2)
        radius = max(round(expected_radius(compressed_optimizer.structure) * 0.6, 2), 1.1)
        relaxed, _ = compressed_optimizer.get_optimized_structure_and_energy(radius=radius)
        return relaxed
    except:
        print("ERROR")
        return -1

C:\Users\dylan\anaconda3\envs\roost\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer VarianceScaling is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [7]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [8]:
df = pd.DataFrame()

In [9]:
df = pd.DataFrame()
df['final_structure'] = structures

In [10]:
df1 = df.sample(frac=0.0001)

In [11]:
print(df1)

                                          final_structure
138286  [[ 2.19346667  1.26885897 13.33781284] Eu, [ 6...
526792  [[ 2.19346667  1.26885897 13.33781284] Yb, [ 6...
491871  [[ 2.19346667  1.26885897 13.33781284] Tm, [ 6...
520693  [[ 2.19346667  1.26885897 13.33781284] Yb, [ 6...
135726  [[ 2.19346667  1.26885897 13.33781284] Eu, [ 6...
102686  [[ 2.19346667  1.26885897 13.33781284] Dy, [ 6...
223760  [[ 2.19346667  1.26885897 13.33781284] Ir, [ 6...
378170  [[ 2.19346667  1.26885897 13.33781284] Re, [ 6...
524562  [[ 2.19346667  1.26885897 13.33781284] Yb, [ 6...
378839  [[ 2.19346667  1.26885897 13.33781284] Re, [ 6...
410696  [[ 2.19346667  1.26885897 13.33781284] Sg, [ 6...
54532   [[ 2.19346667  1.26885897 13.33781284] Ce, [ 6...
122139  [[ 2.19346667  1.26885897 13.33781284] Er, [ 6...
395294  [[ 2.19346667  1.26885897 13.33781284] Rh, [ 6...
158037  [[ 2.19346667  1.26885897 13.33781284] Gd, [ 6...
203943  [[ 2.19346667  1.26885897 13.33781284] Ho, [ 6...
331283  [[ 2.1

In [12]:
df1['final_structure'] = df1['final_structure'].progress_apply(relaxStructure)

  0%|          | 0/55 [00:00<?, ?it/s]

1/1 [==============================] - 0s 33ms/step


Isolated atoms found in the structure. The cutoff radius might be small


1/1 [==============================] - 0s 37ms/step


Isolated atoms found in the structure. The cutoff radius might be small


...

1/1 [==============================] - 0s 60ms/step


Isolated atoms found in the structure. The cutoff radius might be small


1/1 [==============================] - 0s 65ms/step


Isolated atoms found in the structure. The cutoff radius might be small


1/1 [==============================] - 0s 66ms/step


In [13]:
df1 = df1[df1["final_structure"] != -1]
print(df1)

                                          final_structure
138286  [[ 4.20570268  4.13338396 10.930311  ] Eu, [1....
526792  [[ 4.96911326  4.2033642  13.22633025] Yb, [ 1...
491871  [[5.10985005e+00 3.93991439e-03 9.54908722e+00...
520693  [[ 5.27592048  0.04031935 13.1617937 ] Yb, [ 1...
102686  [[ 4.42368224  3.88936637 11.35360732] Dy, [ 0...
223760  [[ 1.44285478  2.14625621 13.33973003] Hf, [6....
378170  [[0.13969252 0.0859676  8.35849766] Ti, [3.033...
524562  [[ 5.17635876  4.32676446 13.04565144] Yb, [6....
378839  [[ 1.18344389  2.11427724 13.31460728] Y, [ 0....
54532   [[4.02213797 0.09411396 8.02509617] Ce, [0.820...
122139  [[ 1.3127025   2.16234033 13.25526076] Y, [ 5....
395294  [[ 0.13887711  0.08688139 11.52516578] Ti, [ 3...
158037  [[ 0.          0.         13.21166539] Gd, [2....
203943  [[4.94866767e+00 4.37807512e-03 1.31605810e+01...
331283  [[ 5.06453636  0.03975381 13.86904642] Pm, [ 1...
319615  [[6.45772295 3.77106612 6.51364362] Ti, [3.434...
453273  [[ 5.0

In [14]:
df1['wyckoff'] = df1['final_structure'].apply(get_aflow_label_spglib)
df1['composition'] = df1['final_structure'].apply(lambda x: x.composition.reduced_formula)

In [15]:
df1 = df1.drop('final_structure', axis=1)

In [16]:
print(df1)

                                                  wyckoff         composition
138286  AB2C2D4E4F_aP14_1_a_2a_2a_4a_4a_a:Eu-Ge-In-S-S...    EuTaIn2Ge2(SeS)4
526792  A2B2C4D4EF_aP14_1_2a_2a_4a_4a_a_a:As-Ga-Po-Se-...  YbTaGa2Po4(AsSe2)2
491871  A2B2C4DE4F_aP14_1_2a_2a_4a_a_4a_a:Bi-Ge-Po-Rh-...     TmPo4Ge2Bi2RhS4
520693  A2BC4D4E2F_aP14_1_2a_a_4a_4a_2a_a:Ga-Nd-Po-Te-...   YbNdTi2Ga2(PoTe)4
102686  A2BC4D4E2F_aP14_1_2a_a_4a_4a_2a_a:As-Dy-S-Se-T...    DyZrTi2As2(SeS)4
223760  ABC4D2E2F4_aP14_1_a_a_4a_2a_2a_4a:Hf-Ir-Po-Sb-...    HfSn2Po4Sb2Te4Ir
378170         A2B8CD2E_aP14_1_2a_8a_a_2a_a:In-Po-Re-Ti-V        Ti2VIn2RePo8
524562  A2B2C4DE4F_aP14_1_2a_2a_4a_a_4a_a:Ga-Ge-Po-Rh-...    YbGa2Po4Ge2RhSe4
378839  A2B2CD4E4F_aP14_1_2a_2a_a_4a_4a_a:Ge-In-Re-Se-...    YIn2ReGe2(TeSe)4
54532   AB2C4DE2F4_aP14_1_a_2a_4a_a_2a_4a:Ce-Ge-Po-Re-...  CeRePo4Ge2(SbSe2)2
122139  A2BC2D4E4F_mP14_6_ab_a_ab_2a2b_2a2b_b:As-Er-In...   YErIn2Te4(AsSe2)2
395294  ABC2D4E4F2_aP14_1_a_a_2a_4a_4a_2a:Fe-Rh-Sb-Se-...    Ti2

In [17]:
df1.to_csv('neuralNetworkInputMBTFamilyRELAXED.csv', index = False, encoding='utf-8')